<a href="https://colab.research.google.com/github/jsilbergDS/CS236project/blob/main/pix2pix_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adapted from: https://github.com/phillipi/pix2pix

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [ ]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [ ]:
!pip install -r requirements.txt

I pair the original highway images with my hand-drawn masks

In [ ]:
!python datasets/combine_A_and_B.py --fold_A /content/drive/MyDrive/BayAreaHighway250Raw_pix --fold_B /content/drive/MyDrive/BayAreaHighway250Mask2 --fold_AB /content/drive/MyDrive/BayAreaHighway250AB_pix

[fold_A] =  /content/drive/MyDrive/BayAreaHighway250Raw_pix
[fold_B] =  /content/drive/MyDrive/BayAreaHighway250Mask2
[fold_AB] =  /content/drive/MyDrive/BayAreaHighway250AB_pix
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  False
split = train, use 100/100 images
split = train, number of images = 100


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

I install the pre-trained model to improve on

In [ ]:
!bash ./scripts/download_pix2pix_model.sh sat2map

I finetune the model with the data pairs

In [ ]:
!python train.py --dataroot /content/drive/MyDrive/BayAreaHighway250AB_pix --name sat2map_pretrained --model pix2pix

# Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

I pair the test set to create the test set masks

In [ ]:
!python datasets/combine_A_and_B.py --fold_A /content/drive/MyDrive/NOLATest --fold_B /content/drive/MyDrive/NOLATest --fold_AB /content/drive/MyDrive/NOLATestAB

[fold_A] =  /content/drive/MyDrive/NOLATest
[fold_B] =  /content/drive/MyDrive/NOLATest
[fold_AB] =  /content/drive/MyDrive/NOLATestAB
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  False
split = test, use 5/5 images
split = test, number of images = 5


I run inference on the test set. I would change this to run inference on 250 unpaired highway images to create the training masks for the "full model" configuration. Within utils I edit:     




```
def save_images(webpage, visuals, image_path, aspect_ratio=1.0, width=256, use_wandb=False):
    """Save images to the disk.

    Parameters:
        webpage (the HTML class) -- the HTML webpage class that stores these imaegs (see html.py for more details)
        visuals (OrderedDict)    -- an ordered dictionary that stores (name, images (either tensor or numpy) ) pairs
        image_path (str)         -- the string is used to create image paths
        aspect_ratio (float)     -- the aspect ratio of saved images
        width (int)              -- the images will be resized to width x width

    This function will save images stored in 'visuals' to the HTML file specified by 'webpage'.
    """
    image_dir = webpage.get_image_dir()
    short_path = ntpath.basename(image_path[0])
    name = os.path.splitext(short_path)[0]

    webpage.add_header(name)
    ims, txts, links = [], [], []
    ims_dict = {}
    for label, im_data in visuals.items():
        im = util.tensor2im(im_data)
        image_name = '%s_%s.png' % (name, label)
        save_path = os.path.join(image_dir, image_name)
        util.save_image(im, save_path, aspect_ratio=aspect_ratio)
        ims.append(image_name)
        txts.append(label)
        links.append(image_name)
        if use_wandb:
            ims_dict[label] = wandb.Image(im)
    webpage.add_images(ims, txts, links, width=width)
    if use_wandb:
        wandb.log(ims_dict)
```

With



```
def save_images(webpage, visuals, image_path, aspect_ratio=1.0, width=256, use_wandb=False):
    """Save images to the disk.

    Parameters:
        webpage (the HTML class) -- the HTML webpage class that stores these imaegs (see html.py for more details)
        visuals (OrderedDict)    -- an ordered dictionary that stores (name, images (either tensor or numpy) ) pairs
        image_path (str)         -- the string is used to create image paths
        aspect_ratio (float)     -- the aspect ratio of saved images
        width (int)              -- the images will be resized to width x width

    This function will save images stored in 'visuals' to the HTML file specified by 'webpage'.
    """
    image_dir = webpage.get_image_dir()
    short_path = ntpath.basename(image_path[0])
    name = os.path.splitext(short_path)[0]

    webpage.add_header(name)
    ims, txts, links = [], [], []
    ims_dict = {}
    for label, im_data in visuals.items():
      if label == "fake_B"
        im = util.tensor2im(im_data)
        image_name = '%s_%s.png' % (name, label)
        save_path = os.path.join(image_dir, image_name)
        util.save_image(im, save_path, aspect_ratio=aspect_ratio)
        ims.append(image_name)
        txts.append(label)
        links.append(image_name)
        if use_wandb:
            ims_dict[label] = wandb.Image(im)
    webpage.add_images(ims, txts, links, width=width)
    if use_wandb:
        wandb.log(ims_dict)
```

To create cleaner folders with only the test masks I need


In [ ]:
!python test.py --dataroot /content/drive/MyDrive/NOLATestAB --model pix2pix --name sat2map_pretrained --checkpoints_dir /content/drive/MyDrive/checkpoints